In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error

import warnings 
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.display import display, clear_output


In [2]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#! jupyter serverextension enable voila --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling: voila
- Writing config: C:\Users\Asus\anaconda3\etc\jupyter
    - Validating...
      voila 0.5.5 ok


In [3]:
file = open('image.jpg', "rb")
image = file.read()

image_headline = widgets.Image(value = image,
                              format = 'jpg',
                              width = '300')

vbox_headline = widgets.VBox([image_headline])

In [4]:
df = pd.read_csv('C:\\Users\\Asus\\Documents\\A_Data\\laptopData.csv')

In [5]:
df1 = df.dropna()

In [6]:
xres = []
yres = []
res = []
touch = []
for i in df1['ScreenResolution']:
    [x,y] = i.split(' ')[-1].split('x')
    if 'Touchscreen' in i:
        touch.append(1)
    else:
        touch.append(0)

    res.append(x+'x'+y)

df1['res'] = res
df1['touch'] = touch


In [7]:
df1['Inches'] = df1['Inches'].replace('?',float('0'))
df1['Inches'] = df1['Inches'].astype('float')


In [8]:
freq = []
proc = []

for i in df1['Cpu']:
    ent = ' '.join(i.split('GHz')[0].split(' ')[:3])
    freq.append(i.split('GHz')[0].split(' ')[-1])
    if 'Intel' in ent:
        if ent[6:] != 'Core i3' and ent[6:] != 'Core i5' and ent[6:] != 'Core i7':
            ent = 'Other Intel Core'
    else:
        ent = 'AMD Core'
    proc.append(ent)
       
df1['proc_freq'] = freq
df1['CPU_Brand'] = proc

In [9]:
ram = []
for i in df1['Ram']:
    ram.append(i[:-2])
    
df1['Ram'] = ram

In [10]:
storage = []
storage_type = []
dual_storage = []
dual_storage_type = []

df1 = df1[df1['Memory'] != '?']

for i in df1['Memory']:
    if '+' not in i:
        if 'TB' not in i.split(' ')[0]:
            storage.append(int(i.split(' ')[0].split('GB')[0]))
        else:
            storage.append(1000*int(float(i.split(' ')[0].split('TB')[0])))
                
        storage_type.append(' '.join(i.split(' ')[1:]))
        dual_storage.append(0)
        dual_storage_type.append(0)
    else:
        if 'TB' not in i.split('+')[0].split(' ')[0]:
            storage.append(int(i.split('+')[0].split(' ')[0].split('GB')[0]))
        else:
            storage.append(1000*int(float(i.split('+')[0].split(' ')[0].split('TB')[0])))
        storage_type.append(' '.join(i.split('+')[0].split(' ')[1:]))
        
        if 'TB' not in i.split('+')[1].split(' ')[2]:
            dual_storage.append(int(i.split('+')[1].split(' ')[2].split('GB')[0]))
        else:
            dual_storage.append(1000*int(float(i.split('+')[1].split(' ')[2].split('TB')[0])))
        dual_storage_type.append(' '.join(i.split('+')[1].split(' ')[3:]))

storage_type = list(map(lambda x: x.replace('HDD ', 'HDD'), storage_type))
storage_type = list(map(lambda x: x.replace('SSD ', 'SSD'), storage_type))
storage_type = list(map(lambda x: x.replace('Flash Storage ', 'Flash Storage'), storage_type))

       
df1['storage'] = storage
df1['storage_type'] = storage_type
df1['dual_storage'] = dual_storage
df1['dual_storage_type'] = dual_storage_type
df1['dual_storage_type'][df1['dual_storage_type'] == 0] = 'None' 

df1['total_storage'] = df1['storage'] + df1['dual_storage']

In [11]:
gpu = []

for i in df1['Gpu']:
    gpu.append(i.split(' ')[0])
    
df1['Gpu'] = gpu

In [12]:
weight =[]

for i in df1['Weight']:
    weight.append(i.split('kg')[0])

df1['Weight'] = weight
df1 = df1[df1['Weight'] != '?']
df1['Weight'] = df1['Weight'].astype('float64')
df1 = df1[df1['Weight'] >= 1]


In [13]:
def subcategory(text):
    if text =='Windows 10' or text =='Windows 7' or text == 'Windows 10 S ':
        return 'Windows'
    elif text =='macOS' or text == 'Mac OS X':
        return 'Mac'
    else:
        return 'Others'
df1['OpSys'] = df1['OpSys'].apply(lambda x:subcategory(x))

In [14]:
df1 = df1.drop(df1.columns[[4,5,7,18,20]],axis = 1)

In [15]:
st = []

for i in df1['storage']:
    if i == '1.0TB':
        st.append('1TB')
        
    else:
        st.append(i)
df1['storage'] = st

df1 = df1[df1['storage'] != '?']
df2 = df1.copy()

# Encoding

In [16]:
df2['Company'] = df1['Company'].astype('category').cat.codes
df2['TypeName'] = df1['TypeName'].astype('category').cat.codes
df2['Gpu'] = df1['Gpu'].astype('category').cat.codes
df2['res'] = df1['res'].astype('category').cat.codes
df2['touch'] = df1['touch'].astype('category').cat.codes
df2['OpSys'] = df1['OpSys'].astype('category').cat.codes
df2['CPU_Brand'] = df1['CPU_Brand'].astype('category').cat.codes
df2['storage_type'] = df1['storage_type'].astype('category').cat.codes
df2['dual_storage_type'] = df1['dual_storage_type'].astype('category').cat.codes

df2['Ram'] = df1['Ram'].astype('int32')
#df2['Weight'] = df1['Weight'].astype('float64')
df2['proc_freq'] = df1['proc_freq'].astype('float64')

# Company

In [17]:
#df2['Company'] = df1.Company.astype('category').cat.categories

dict1 = dict(enumerate(df1.Company.astype('category').cat.categories))
dict2 = {}

dict2 = dict([(v, [k for k, v1 in dict1.items() if v1 == v])
              for v in set(dict1.values())])
dict2 = {key: int(value[0]) for key, value in dict2.items()}


company_wid = widgets.Dropdown(
    options=list(dict2.items()),
    description='Brand:',
)

# Laptop Type

In [18]:
#df2['TypeName'] = df1.TypeName.astype('category').cat.categories
dict1 = dict(enumerate(df1.TypeName.astype('category').cat.categories))
dict2 = {}

dict2 = dict([(v, [k for k, v1 in dict1.items() if v1 == v])
              for v in set(dict1.values())])

dict2 = {key: int(value[0]) for key, value in dict2.items()}

lap_type_wid = widgets.Dropdown(
    options=list(dict2.items()),
    description='Type:',
)

# Screen Size

In [19]:
screen_size = list(np.sort(df1['Inches'].unique()))[1:]

screen_size_wid= widgets.Dropdown(
    options=screen_size,
    description='Screen Size:',
)

# RAM

In [20]:
ram = list(np.sort(df1['Ram'].unique().astype('int')))

ram_wid = widgets.Dropdown(
    options=ram,
    description='Ram:',
)

# GPU

In [21]:
#df2['Gpu'] = df1.Gpu.astype('category').cat.categories

dict1 = dict(enumerate(df1.Gpu.astype('category').cat.categories))
dict2 = {}

dict2 = dict([(v, [k for k, v1 in dict1.items() if v1 == v])
              for v in set(dict1.values())])
dict2 = {key: int(value[0]) for key, value in dict2.items()}

gpu_wid = widgets.Dropdown(
    options=list(dict2.items()),
    description='GPU:',
)

# Operating System

In [47]:

dict1 = dict(enumerate(df1.OpSys.astype('category').cat.categories))
dict2 = {}

dict2 = dict([(v, [k for k, v1 in dict1.items() if v1 == v])
              for v in set(dict1.values())])
dict2 = {key: int(value[0]) for key, value in dict2.items()}

op_sys_wid = widgets.ToggleButtons(
    options=list([('Windows', 2), ('Mac', 0), ('Others', 1)]),
    description='OS:',
    disabled=False,
    button_style='info',
     icons=['fa-brands fa-windows', 'fa-brands fa-apple', ' ']
)

# Weight

In [23]:

weight_wid = widgets.BoundedFloatText(
    min=0,
    max=9,
    step=0.1,
    description='Weight (kgs):',
    disabled=False
)

# Touchscreen

In [24]:

touch_wid = widgets.RadioButtons(
    options=[('Yes', 1), ('No', 0)],
    description='Touchscreen:',
    disabled=False
)

# Resolution

In [25]:
#df2['res'] = 
dict1 = dict(enumerate(df1.res.astype('category').cat.categories))
dict2 = {}

dict2 = dict([(v, [k for k, v1 in dict1.items() if v1 == v])
              for v in set(dict1.values())])
dict2 = {key: int(value[0]) for key, value in dict2.items()}

resol_wid = widgets.Dropdown(
    options=list(dict2.items()),
    description='Resolution:',
)

# CPU Frequency

In [26]:

freq_wid = widgets.FloatSlider(
    min=0.9,
    max=3.6,
    step=0.1,
    description='CPU Frequency:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)


# CPU Brand

In [27]:

dict1 = dict(enumerate(df1.CPU_Brand.astype('category').cat.categories))
dict2 = {}

dict2 = dict([(v, [k for k, v1 in dict1.items() if v1 == v])
              for v in set(dict1.values())])
dict2 = {key: int(value[0]) for key, value in dict2.items()}

cpu_wid = widgets.ToggleButtons(
    options=list(dict2.items()),
    description='CPU Brand:',
    disabled=False,
    button_style='info'
)

# Storage

In [28]:
store = list(np.sort(df1['storage'].unique())) # SelectionSlider

store_wid = widgets.Dropdown(
    options=store,
    description='Storage:',
)

# Storage Type

In [29]:

dict1 = dict(enumerate(df1.storage_type.astype('category').cat.categories))
dict2 = {}

dict2 = dict([(v, [k for k, v1 in dict1.items() if v1 == v])
              for v in set(dict1.values())])
dict2 = {key: int(value[0]) for key, value in dict2.items()}

store_type_wid = widgets.ToggleButtons(
    options=list(dict2.items()),
    description='Data Drive:',
    disabled=False,
    button_style='info'
)

#  Dual Storage

In [30]:

dict1 = dict(enumerate(df1.dual_storage_type.astype('category').cat.categories))
dict2 = {}

dict2 = dict([(v, [k for k, v1 in dict1.items() if v1 == v])
              for v in set(dict1.values())])
dict2 = {key: int(value[0]) for key, value in dict2.items()}

dual_store_wid = widgets.ToggleButtons(
    options=list(dict2.items()),
    description='Dual Storage',
    button_style='info'
)

In [31]:
x = df2[['Company', 'TypeName', 'Inches','Ram', 'Gpu', 'OpSys', 'Weight', 'res', 'touch', 
         'proc_freq', 'CPU_Brand', 'storage', 'storage_type', 'dual_storage_type']]
y=df2['Price']

In [32]:
x_train, x_test, y_train, y_test = tts(x,y, test_size= 0.2, random_state=101)

In [33]:
from sklearn.preprocessing import StandardScaler

sc=StandardScaler()
x_train_s=sc.fit_transform(x_train)
x_test_s=sc.transform(x_test)

In [34]:
model = RandomForestRegressor()
model.fit(x_train_s, y_train)

RandomForestRegressor()

In [35]:
#pred_rand = model.predict(x_test_s)

# Prediction Button

In [36]:
#from sklearn.metrics import accuracy_score, r2_score

#r2_score(y_test,pred_rand)

In [37]:
button_pred = widgets.Button(
                description='Predict the Price',
                style={'description_width': 'initial'}
            )


output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        print(f"The predicted value of the laptop you are looking for is ₹{np.round(model.predict([[company_wid.value, lap_type_wid.value, screen_size_wid.value, ram_wid.value, gpu_wid.value, op_sys_wid.value, weight_wid.value, resol_wid.value, touch_wid.value, freq_wid.value, cpu_wid.value, store_wid.value, store_type_wid.value, dual_store_wid.value]])[0],2)}")

button_pred.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_pred, output])

In [49]:
vbox_text = widgets.VBox([company_wid,lap_type_wid, screen_size_wid, ram_wid, 
                          gpu_wid, op_sys_wid, weight_wid, resol_wid, touch_wid, freq_wid,
                          cpu_wid, store_wid, store_type_wid, dual_store_wid, vbox_result])


In [50]:
page = widgets.HBox([vbox_headline, vbox_text])
display(page)